## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

http://www.pitt.edu/~naraehan/presentation/word2vec-try.html
https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34

In [1]:
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))
lines_test = list(open('./news_test.txt', 'r', encoding='utf-8'))
len(lines), len(lines_test)

(15000, 3000)

In [2]:
import random
random.shuffle(lines)

In [3]:
type_train = [line.split('\t')[0] for line in lines]
head_train = [line.split('\t')[1] for line in lines]
text_train = [line.split('\t')[2] for line in lines]

In [4]:
type_test = [line.split('\t')[0] for line in lines_test]
head_test = [line.split('\t')[1] for line in lines_test]
text_test = [line.split('\t')[2] for line in lines_test]

In [5]:
from collections import Counter

Counter([line.split('\t')[0] for line in lines[:15000]]),\
Counter([line.split('\t')[0] for line in lines_test])

(Counter({'forces': 1225,
          'culture': 2053,
          'life': 2033,
          'media': 2111,
          'science': 2156,
          'sport': 2215,
          'economics': 2080,
          'travel': 289,
          'style': 284,
          'business': 554}),
 Counter({'culture': 426,
          'media': 403,
          'business': 90,
          'life': 415,
          'science': 466,
          'forces': 245,
          'sport': 423,
          'economics': 426,
          'travel': 54,
          'style': 52}))

1. Обработать данные, получив для каждого текста набор токенов

In [6]:
import nltk
from nltk.tokenize import sent_tokenize
# nltk.download('punkt')
# nltk.download("stopwords")

In [7]:
from nltk.stem.snowball import RussianStemmer 
from nltk.corpus import stopwords
from string import punctuation

russian_stopwords = stopwords.words("russian")
russian_stopwords += ['что', 'это', 'так', 'вот', 'быть', '—', '–', 'к',  '...']
rs = RussianStemmer()

def preprocess_text(text):
    symbols = "«»0123456789!\"#$%&()*+./:;<=>?@[\]^_`{|}~=\n,-"
    for j in symbols:
        text = text.replace(j, ' ')

    tokens = [token for token in text.lower().split() if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]

    tokens = [rs.stem(k) for k in tokens]

    return tokens

In [8]:
text_train_tokens = []
for text in text_train:
    text_train_sent = sent_tokenize(text)
    text_tokens = []
    for sent in text_train_sent:
        text_tokens += preprocess_text(sent) 
    text_train_tokens += [text_tokens]

In [9]:
text_train_proc = [' '.join(i) for i in text_train_tokens]

In [10]:
text_train_tokens[0]

['петрозаводск',
 'городск',
 'суд',
 'взял',
 'страж',
 'заместител',
 'директор',
 'парк',
 'отел',
 'сямозер',
 'вадим',
 'виноградов',
 'переда',
 'интерфакс',
 'отмет',
 'суд',
 'сво',
 'решен',
 'арест',
 'виноград',
 'продолж',
 'занима',
 'преступн',
 'деятельн',
 'скрыт',
 'следств',
 'оказа',
 'давлен',
 'свидетел',
 'гибел',
 'дет',
 'прощ',
 'заяв',
 'подсудим',
 'арестова',
 'месяц',
 'август',
 'переда',
 'ри',
 'новост',
 'врем',
 'суд',
 'отказа',
 'взят',
 'страж',
 'вожат',
 'валер',
 'круподерщиков',
 'сообщ',
 'тасс',
 'суд',
 'наход',
 'достаточн',
 'закон',
 'основан',
 'удовлетворен',
 'ходатайств',
 'избран',
 'отношен',
 'подозрева',
 'круподерщиков',
 'сам',
 'строг',
 'мер',
 'пресечен',
 'сказа',
 'суд',
 'александр',
 'смирн',
 'постанов',
 'освобод',
 'вожат',
 'зал',
 'суд',
 'ран',
 'июн',
 'арестова',
 'директор',
 'парк',
 'отел',
 'сямозер',
 'ел',
 'решетов',
 'подозрева',
 'халатн',
 'повлекш',
 'смерт',
 'двух',
 'лиц',
 'такж',
 'оказан',
 'услуг'

In [11]:
text_test_tokens = []
for text in text_test:
    text_test_sent = sent_tokenize(text)
    text_tokens = []
    for sent in text_test_sent:
        text_tokens += preprocess_text(sent) 
    text_test_tokens += [text_tokens]

In [12]:
text_test_proc = [' '.join(i) for i in text_test_tokens]

2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать gensim . Продемонстрировать семантические ассоциации.

In [13]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=text_train_tokens, 
                 window=5, 
                 min_count=5, 
                 workers=4)
model.save("word2vec.model")

In [14]:
# model = Word2Vec(sentences=text_train_tokens, 
#                  min_count=5,
#                  window=5,
#                  size=300,
#                  sample=6e-5, 
#                  alpha=0.03, 
#                  min_alpha=0.0007, 
#                  negative=20,
#                  workers=4)

In [15]:
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)

Word2Vec(vocab=25344, size=100, alpha=0.025)
['петрозаводск', 'городск', 'суд', 'взял', 'страж', 'заместител', 'директор', 'парк', 'отел', 'сямозер', 'вадим', 'виноградов', 'переда', 'интерфакс', 'отмет', 'сво', 'решен', 'арест', 'виноград', 'продолж', 'занима', 'преступн', 'деятельн', 'скрыт', 'следств', 'оказа', 'давлен', 'свидетел', 'гибел', 'дет', 'прощ', 'заяв', 'подсудим', 'арестова', 'месяц', 'август', 'ри', 'новост', 'врем', 'отказа', 'взят', 'вожат', 'валер', 'сообщ', 'тасс', 'наход', 'достаточн', 'закон', 'основан', 'удовлетворен', 'ходатайств', 'избран', 'отношен', 'подозрева', 'сам', 'строг', 'мер', 'пресечен', 'сказа', 'александр', 'смирн', 'постанов', 'освобод', 'зал', 'ран', 'июн', 'ел', 'решетов', 'халатн', 'повлекш', 'смерт', 'двух', 'лиц', 'такж', 'оказан', 'услуг', 'отвеча', 'требован', 'безопасн', 'материал', 'тем', 'урок', 'произошл', 'озер', 'рамк', 'расследован', 'уголовн', 'дел', 'задержа', 'руководител', 'республиканск', 'управлен', 'роспотребнадзор', 'анатол',

In [16]:
model.wv.vocab

{'петрозаводск': <gensim.models.keyedvectors.Vocab at 0x19f50d59d30>,
 'городск': <gensim.models.keyedvectors.Vocab at 0x19f50d59e10>,
 'суд': <gensim.models.keyedvectors.Vocab at 0x19f50d59e48>,
 'взял': <gensim.models.keyedvectors.Vocab at 0x19f50d59e80>,
 'страж': <gensim.models.keyedvectors.Vocab at 0x19f50d59eb8>,
 'заместител': <gensim.models.keyedvectors.Vocab at 0x19f50d59ef0>,
 'директор': <gensim.models.keyedvectors.Vocab at 0x19f50d59f28>,
 'парк': <gensim.models.keyedvectors.Vocab at 0x19f50d59f60>,
 'отел': <gensim.models.keyedvectors.Vocab at 0x19f50d59f98>,
 'сямозер': <gensim.models.keyedvectors.Vocab at 0x19f50d59fd0>,
 'вадим': <gensim.models.keyedvectors.Vocab at 0x19f50d78048>,
 'виноградов': <gensim.models.keyedvectors.Vocab at 0x19f50d78080>,
 'переда': <gensim.models.keyedvectors.Vocab at 0x19f50d780b8>,
 'интерфакс': <gensim.models.keyedvectors.Vocab at 0x19f50d780f0>,
 'отмет': <gensim.models.keyedvectors.Vocab at 0x19f50d78128>,
 'сво': <gensim.models.keyedvec

In [17]:
model.wv.most_similar(positive=['июл'], negative=['shell'], topn=12)

[('ма', 0.6639769673347473),
 ('ноябр', 0.6615877151489258),
 ('август', 0.6602712869644165),
 ('июн', 0.6593568325042725),
 ('октябр', 0.6489208936691284),
 ('апрел', 0.645183801651001),
 ('декабр', 0.6444573998451233),
 ('март', 0.6428337097167969),
 ('феврал', 0.6366623640060425),
 ('сентябр', 0.6314038634300232),
 ('январ', 0.6283018589019775),
 ('поздн', 0.5584112405776978)]

In [18]:
model.wv.most_similar(positive=['торговл', 'крым'], negative=['shell'], topn=10)

[('регион', 0.7286562919616699),
 ('полуостров', 0.7174791693687439),
 ('украин', 0.7017306685447693),
 ('киев', 0.6914758682250977),
 ('транспорт', 0.688478946685791),
 ('таможен', 0.6815627813339233),
 ('турц', 0.6805194616317749),
 ('республик', 0.6775258183479309),
 ('присоединен', 0.6709744930267334),
 ('туризм', 0.6688839197158813)]

In [19]:
# from sklearn.decomposition import PCA
# from matplotlib import pyplot

# fig = plt.figure()
# fig.set_size_inches(18.5, 10.5)
# X = model.wv[model.wv.vocab]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)
# pyplot.scatter(result[:, 0], result[:, 1])

# for i, word in enumerate(text_train_tokens[2]):
#     pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
# pyplot.show()

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [21]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(type_train)
Test_Y = Encoder.fit_transform(type_test)

In [22]:
Tfidf_vect = TfidfVectorizer(max_features=15000)
Tfidf_vect.fit(text_train_proc)
Train_X_Tfidf = Tfidf_vect.transform(text_train_proc)
Test_X_Tfidf = Tfidf_vect.transform(text_test_proc)

In [23]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Test_Y) * 100)

Naive Bayes Accuracy Score ->  83.6


In [24]:
logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
logit.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation dataset
predictions_logit = logit.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Logistic regression Accuracy Score -> ", accuracy_score(predictions_logit, Test_Y) * 100)

Logistic regression Accuracy Score ->  88.86666666666667


In [29]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf, Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y) * 100)

SVM Accuracy Score ->  88.56666666666668
